# Case Study: Nested Data Structures

As an example of how Python makes it easy to mix and match data structures, we'll look at an example of a complex, nested data structure &mdash; an ontology from the [Allen Brain Atlas](https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph).

In [ ]:
import requests  
url = "http://api.brain-map.org/api/v2/structure_graph_download/1.json"
response = requests.get(url)

Here, `requests` is the module (library) that lets us make requests over HTTP and `url` is the link for the Mouse Brain Atlas. When we call `requests.get` (the `get` function from the `requests` module), we get an HTTP response that we save to a variable of the same name.

If we asked for `type(response)` we could see that it's a `requests.models.Response` object. That shouldn't mean anything to you, but if you look at the `text` attribute of this data object, that looks a little more like something we understand:

In [ ]:
print(type(response.text))
print(response.text[:500])

This string, as the `json` suffix in the filename implies, is a Javascript Object Notation ([JSON](https://en.wikipedia.org/wiki/JSON)) file. JSON is a very common format for structured information used on the web. It's both hierarchical and human-readable, which makes it convenient for flexibly organizing data. 

Fortunately, Python has a built-in module called, fittingly, `json`, for reading such strings and turning them into sensibly Python objects:

In [ ]:
import json

dat = json.loads(response.text)

print(type(dat))

As we can see, `json.loads` ("load string") has turned our JSON object into a `dict`, and since we know how to deal with dictionaries, we can explore it.

In [ ]:
dat.keys()

If we take turns looking at the values corresponding to these keys, we pretty quickly see that all the action is in the `msg` entry:

In [ ]:
dat['msg']

This seems to be a list, and indeed it is:

In [ ]:
print(f"Type of dat['msg'] is {type(dat['msg'])}.")
print(f"Length of dat['msg'] is {len(dat['msg'])}.")

```{tip}
Here, we've used a new way of writing strings &mdash; the [f-string](https://docs.python.org/3/tutorial/inputoutput.html). F-strings (available in newer versions of Python) allow you to simply write the variables you want interpolated into the string as Python expressions inside braces (`{}`). This is a handy format for printing helpful text, but **note that strings must be preceded by `f"`!**
```

Now, a list with one element is not very exciting, but this is a _hierarchical_ data structure (we suspect), so we can explore what happens as we dig deeper.

In [ ]:
# not a good variable name, but it will save typing here
m = dat['msg']  

print(f"m[0] is of type {type(m[0])}.")
m[0]

If we look closely, we can see that `m[0]['name']` is `root`, suggesting it's the top node of a tree. And there are some other entries here, like the `ontology_id` that might be important, but the interesting stuff appears to be in `children`, which looks like a list. So let's try:

In [ ]:
print(f"m[0]['children'] has {len(m[0]['children'])} entries.")

Note that, unlike some languages, we can write expressions like `m[0]['children']` with no problem. Python treats this sensibly by reading left-to-right: get the 0th entry from `m`, then get the `'children'` entry from the result. And we can use another trick we learned in the last section to find out what the children of the root node are:

In [ ]:
[c['name'] for c in m[0]['children']]

Aha! This looks like a breakdown of all the labeled structures into top-level categories. And in the same way, we could dig further down:

In [ ]:
basic = m[0]['children'][0] 
[c['name'] for c in basic['children']]

So what's this good for? Here, we are mostly exploring the way the Allen Brain Atlas is organized. But more generally, we've seen how to download data from the web, turn it into a hierarchical JSON data structure, and explore its contents. That can be useful when we need to, for instance, pull information from one or several online web pages or databases, loop over the results, and extract key pieces of information. Even more impressive, this has only take a few lines of code. The most time-consuming part in this (and most) cases came down to figuring out how to navigate nested lists and dictionaries to find where the information we needed was stored.